# Multilingual Embedding-based Machine Translation

Работа выполнена в рамках курса естественно-научного содержания (ЕНС) _**"Элементы машинного обучения"**_  
Преподаватель: Шокуров Антон Вячеславович

Автор: Хритова Екатерина Андреевна  
Группа: **510**

**Содержание**
> [Общие сведения](#section-1)   
>> [Данные](#subsection-11)  
>> [Мера качества](#subsection-12)  
>> [Embeding](#subsection-13)  

> [Mодель №1](#section-2)  
> [Модель №2](#section-3)   
> [Модель №3](#section-4)   

> [Результаты](#section-5)   

Этот блокнот основан на статьях:  
[1] Samuel L. Smith, David H. P. Turban, Steven Hamblin & Nils Y. Hammerla, [Offline bilingual word vectors, orthogonal transformations and
the inverted softmax](https://openreview.net/pdf?id=r1Aab85gg)   
[2] A. Conneau*, G. Lample*, L. Denoyer, MA. Ranzato, H. Jégou, [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf)  
[3] Armand Joulin Piotr Bojanowski Tomas Mikolov, [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745.pdf)

Рассмотрим задачу машинного перевода. Пусть задано представление слов (как векторов $x \in \mathbb{R}^d$). Для рещения задачи будем выравнивать векторы из двух языков в едином векторном пространстве с помощью некоторого отображения $W$. Ниже представлена схема выравнимания для векторного представления английских слов $X$ и испанских слов $Y$.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

Рассмотрим следующие модели:
1. Использующая линейное отображение между непрерывными представлениями слов.
2. Использующая линейное ортогональное отображение между непрерывными представлениями слов.
3. Использующая линейное ортогональное отображение между непрерывными представлениями слов, при построении которого используется корректирующая метрика.


<a name="section-1"></a> 
# Общие сведения <a name="subsection-11"></a> 

Для исследования выберем два родственных славянских языка: **украинский** и **русский**. 

<a name="subsection-13"></a> 
## Embeding

Для embeding-а возьмем уже готовые представления:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Посмотрим на слова, наиболее близкие к слову _"август"_ и _"серпень"_ (перевод на украинский язык)

In [1]:
import pandas as pd
from gensim.models import KeyedVectors

In [2]:
# EMBEDING
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [3]:
ruemb_to_ru = ru_emb.most_similar([ru_emb["август"]])
#ukemb_to_ro = uk_emb.most_similar([ru_emb["август"]])
ukemb_to_uk = uk_emb.most_similar([uk_emb["серпень"]])
ruemb_to_uk = ru_emb.most_similar([uk_emb["серпень"]])

print('Наиболее близкие к слову "август" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_ru[k][0], round(ruemb_to_ru[k][1], 3)))
    
print('\nНаиболее близкие к слову "серпень" для uk_emb:')
for k in range(10):
    print("{:10} {}".format(ukemb_to_uk[k][0], round(ukemb_to_uk[k][1], 3)))

print('\nНаиболее близкие к слову "серпень" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_uk[k][0], round(ruemb_to_uk[k][1], 3)))

Наиболее близкие к слову "август" для ru_emb:
август     1.0
июль       0.938
сентябрь   0.924
июнь       0.922
октябрь    0.91
ноябрь     0.893
апрель     0.873
декабрь    0.865
март       0.855
февраль    0.84

Наиболее близкие к слову "серпень" для uk_emb:
серпень    1.0
липень     0.91
вересень   0.902
червень    0.899
жовтень    0.881
листопад   0.879
квітень    0.859
грудень    0.859
травень    0.841
лютий      0.826

Наиболее близкие к слову "серпень" для ru_emb:
Недопустимость 0.244
конструктивность 0.233
офор       0.233
deteydlya  0.23
пресечении 0.226
одностороннего 0.226
подход     0.223
иболее     0.22
2015Александр 0.219
конструктивен 0.218


## Данные

В качестве исходного датасета используется файл, содержащий украинские и русские слова, разделенные табуляцией. Все буквы в словах записаны в нижнем регистре, слова не содаржат знаков препинания, цифр и пр.  
Пример нескольких таких пар:

|Ukrainian  | Russian   |
|-----------|-----------|
|автомобіль | автомобиль|
|автомобіль | вагон     |
|агов       | эй        |
|аґрус      |крыжовник  |
|адже       |ведь
|...        | ...       |

Загрузка данных

In [4]:
import numpy as np

In [5]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [6]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus_train_fixed.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus_test_fixed.txt")

<a name="subsection-12"></a>  
## Мера качества

В качестве меры качества будем ипользовать долю "близко угаданных" слов с точностью top-1, top-5 и top-10:

Для каждого преобразованного украинского представления ищется его N ближайших соседей в пространстве представления русских слов (т.е. слово переводится с украинского на русский и для полученного перевода ищется N наиболее близких по смыслу русских слов). Далее находится частота "близко угаданных" слов, т.е. тех, у которых хотя бы одна точка в найденной окрестности совпадает с истинным переводом.

Аргументы функции:
* Массив _pairs_ содержит пары слов из словаря (т.е. слово и его точный перевод)
* Массив _mapped_vectors_ - это массив построенных приближений/переводов дли всех украинских слов из массива _pairs_
* topn - точность (количество элементов в окрестности)

In [7]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0.
    for i, (_, ru) in enumerate(pairs):
        neighbours = ru_emb.most_similar(positive=[mapped_vectors[i]], topn=topn)
        for neigh, _ in neighbours:
                if neigh == ru:
                    num_matches += 1.
    precision_val = num_matches / len(pairs)
    print(precision_val)
    return precision_val

<a name="section-2"></a>
# Модель №1

Пусть $x_i \in \mathbb{R}^d$ — представление слова $i$ на исходном языке, а $y_i \in \mathbb{R}^d$ — представление его перевода. Наша цель — узнать такое линейное преобразование $W$, которое минимизирует евклидово расстояние между $Wx_i$ и $y_i$ для некоторого подмножества представлений слов. Таким образом, мы можем сформулировать так называемую проблему Прокруста:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$

Заметим, что $W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ выглядит как простая множественная линейная регрессия (без перехвата). Итак, давайте искать ее именно в таком виде.

In [8]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

#Y_pred = mapping.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Посмотрим на соседей вектора слова _"сiчень"_ после линейного преобразования.

In [9]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ruemb_to_uk = ru_emb.most_similar(august)

print('Наиболее близкие к слову "серпень" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_uk[k][0], round(ruemb_to_uk[k][1], 3)))

Наиболее близкие к слову "серпень" для ru_emb:
апрель     0.854
июнь       0.841
март       0.84
сентябрь   0.836
февраль    0.833
октябрь    0.831
ноябрь     0.828
июль       0.824
август     0.812
декабрь    0.804


Таким образом, окрестность этого вложения состоит из разных месяцев, но правильный вариант находится на девятом месте.

Т.е., если мы вычислим меру качества модели для выборки, состоящей только из слова _август_ (и его перевода), то оценка с точность top-5 даст 0.0, а оценка с точностью top-9 и выше даст 1.0

In [10]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0.0
1.0
1.0


Вычислим теперь меру качества построенной модели для тестовой выборки.

In [11]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)
precision_top10 = precision(uk_ru_test, mapping.predict(X_test), 10)

0.6424870466321243
0.8082901554404145
0.8497409326424871


<a name="section-3"></a>
# Модель №2

Можно показать (см. оригинальную статью), что самосогласованное линейное отображение между семантическими пространствами должно быть ортогональным.
Тогда можно искать отображение $W$ как ортогональное:

$$W^*= \arg\min_W ||WX - Y||_2 ,\\ W^TW = I, \\I \text{- identity matrix}$$

Оптимальное ортогональное преобразование можно найти, используя разложение по сингулярным значениям. Оказывается, оптимальное преобразование $W^*$ можно выразить через компоненты SVD:

$$X^TY=U\Sigma V^T\text{, сингулярное разложение}$$
$$W^*=UV^T$$

In [12]:
#singular value decompostion

def learn_transform(X_train, Y_train):
    U, S, V = np.linalg.svd(np.matmul(X_train.T, Y_train))
    return np.matmul(U,V)

In [13]:
W = learn_transform(X_train, Y_train)

In [14]:
prescision_top1 = precision(uk_ru_test, np.matmul(X_test, W), 1)
prescision_top5 = precision(uk_ru_test, np.matmul(X_test, W), 5)
prescision_top10 = precision(uk_ru_test, np.matmul(X_test, W), 10)

0.655440414507772
0.8238341968911918
0.8523316062176166


<a name="section-4"></a>
# Модель №3

Можно показать, что построенные модели страдают от так называемой "hubness problem": некоторые векторы слов имеют тенденцию быть ближайшими соседями аномально большого количества других слов. Одним из решений этой проблеммы является использование при выводе корректирующей метрики. В [[3]](https://arxiv.org/pdf/1804.07745.pdf) в качестве такой метрики выбирается CSLS-критерий:
$$CSLS(x,y) = -2 cos(x,y) + \frac{1}{k}\sum_{y' \in \mathcal{N}_Y(x)} + \frac{1}{k}\sum_{x' \in \mathcal{N}_X(y)} $$

Здесь $\mathcal{N}_Y(x)$ - окрестность точки $x$ в пространстве $Y$ (k ближайших соседей) и $cos(x,y)$ - cosine similarity (косинусное сходство)

Отображение $W$, как и в предыдущей модели, будем искать как ортогональное $W \in \mathcal{O}_d$.

Задача оптимизации (_Relaxed CSLS loss_) в этом случае записывается как
$$W^* = \underset{W \in \mathcal{O}_d}{argmin}(\frac{1}{n}\sum_{i=1}^{n} -2x_i^TW_i^Ty_i + \frac{1}{k}\sum_{y_j\in\mathcal{N}_Y(Wx_i)}x_i^TW^Ty_j +  \frac{1}{k}\sum_{Wx_j\in\mathcal{N}_X(y_i)}x_j^TW^Ty_i)$$

In [17]:
def RCSLS(X, Y, W, k = 10):
    n = X.shape[0]
    X_trans = np.matmul(X, W.T)
    # first term
    f1 = 2*np.sum(X_trans * Y)
    df1 = 2 * np.matmul(Y.T, X)
    #second term
    f2, df2 = knn_term(np.matmul(X_trans, Y.T), X, Y, k)
    #third term
    f3, df3 = knn_term(np.matmul(np.matmul(X, W.T), Y.T).T, Y, X, k)
    
    f = -f1 + f2 + f3
    df = -df1 + df2 + df3.T
    
    return f/n, df/n

В терминах нашей задачи:
$$cos(Wx_i, y_j) = x_i^TW^Ty_j$$
В то же время
$$||Wx_i - y_j||_2^2 = 2 - 2x_i^TW^Ty_j$$

Другими словами, $k$ ближайщих соседей можно искать как $k$ элементов, для которых значение $x^TW^Ty$ будет максимальным.

In [18]:
def knn_term(term_val, space_pnt, space_neigh, k):
    neigh_pid = np.argpartition(term_val, -k, axis=1)[:, -k:]
    shape = neigh_pid.shape
    neigh = space_neigh[neigh_pid.flatten(), :]
    neigh = neigh.reshape(shape[0], shape[1], space_neigh.shape[1])
    
    f = np.sum(term_val[np.arange(term_val.shape[0])[:, None], neigh_pid])
    df = np.matmul(neigh.sum(1).T, space_pnt)
    
    return f/k, df/k

Зададим некоторые параметры

In [37]:
# number of subsample elements
subsetsize = 1000
# number of iterations
niter = 25
# number of nearest neighbors
k = 20
# learning rate
lr = 1.0

In [38]:
#singular value decompostion
W = learn_transform(X_train, Y_train)

X = X_train.copy()
Y = Y_train.copy()

f_old = 0
W_old = []

for it in range(0, niter + 1):
    if lr < 1e-4:
        break
    ids = np.random.choice(X.shape[0], size=subsetsize, replace=False)
    f, df = RCSLS(X[ids, :], Y[ids, :], W, k)      
    W -= lr * df 
    f_old, W_old = f, W

In [39]:
prescision_top1 = precision(uk_ru_test, np.matmul(X_test, W), 1)
prescision_top5 = precision(uk_ru_test, np.matmul(X_test, W), 5)
prescision_top10 = precision(uk_ru_test, np.matmul(X_test, W), 10)

0.6606217616580311
0.8341968911917098
0.8549222797927462


<a name = "section-5"></a>
# Результаты

Оценки, полученные для каждой модели, приведены в следующей таблице:

|      | Модель №1 | Модель №2 | Модель №3 |
|------|-----------|-----------|-----------|
|top-1 | 0.6425    | 0.6554    | 0.6606    |
|top-5 | 0.8083    | 0.8238    | 0.8342    |
|top-10| 0.8497    | 0.8523    | 0.8549    |


Как и ожидалось, модель №1 является худшей, а модель №3 показывает лучший результат.